<a href="https://colab.research.google.com/github/hoangviet-nguyen/bevri-abschluss-projekt/blob/main/projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Under Water Image Segmentation